# cbox_token 받아오기

In [1]:
from bot import NaverBot

bot = NaverBot(id='ds778612', pw='Timlee230.')
bot.login_naver()
cookies = bot.naver_cookie
cookies

defaultdict(str,
            {'PM_CK_loc': '42e236690a3f3f48c140f7ca685eee751edecd0b84188b54933c2058b84c4c52',
             'NID_JKL': 'wtAgAkkaymcgnCpkDNou18PkRt94bwrS6bS3aJEiVOg=',
             'nid_inf': '290551059',
             'NID_SES': 'AAABpWOR9+xoCMVeBzEox4yVgMhGmzFSIjAbzdQ3SVtNSYcdZrTsJTqELz7cjaiQ4j5iEWrPBZFqyIPZAw0V/Ox7+7+DWwVQJzqIEpMxiLw3YwVPMl6DwBmgK4JXzrQF4x3wT0c3dW6/4CHxAwF1W3SF5doMQNNPSRbshVg+BTMgVPvrrBKml4nytf6DUNX4qyRmq8lOE1ueV0+NGBK73cH3L2/uj7PMoJVERavpPNB3l0VUtsKWWqm+NsXETmSoP3Ojc59kvXNtySgEA85liFwH9IP24msnQSNzJEo1Ove9ZyBW2Cn6TJIwizbZQWc9wvUWEGZjYLo/UrBImNp2OoCV5kql0/rM9w06TDCwnFUccafPO5pqcWSMKUgfhbgPIfNo7Ew6ksDGIGI+hl0jGrJF4azKKtT+qt3A6oo1NnVEC+j1agzWmamEk0Ka6/MphN0D9knz4yqyjs9vHeUyxohKbEZO54kTCYW5joh6IA1lm1IB+oVQ0SPJJ1GxgAbjwhv3CI9C+tl9rYpYD3Jla3ZrOl4BJIVb90glzxj3kliUhAk/ZzFaO3OqaBDt5xc1/Vrl/g==',
             'NACT': '1',
             'NID_AUT': 'iVING/VDzpZoDQz1fQ3obEonBhSpFLjKyBZYOlTqZ1PLmbojQCiSij2L5uY2B6Wp',
             'NNB': 'NLLON6TOUULWO',
             

In [30]:
import requests
import re

def _get_cbox_token(cookies, blog_post_id):
    try:
        # Define the URL
        url = "https://apis.naver.com/commentBox/cbox/web_naver_token_jsonp.json"

        # Define the query parameters
        params = {
            'ticket': 'blog',
            'templateId': 'default',
            'pool': 'blogid',
            'lang': 'ko',
            'country': '',
            'objectId': f'154352947_201_{blog_post_id}',
            '_cv': '20240923174033',
            'categoryId': '',
            'groupId': '154352947',
        }

        # Define the headers
        headers = {
            'accept': '*/*',
            'accept-encoding': 'gzip, deflate, br, zstd',
            'accept-language': 'ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7',
            'referer': 'https://m.blog.naver.com/CommentList.naver',
            'sec-ch-ua': '"Google Chrome";v="129", "Not=A?Brand";v="8", "Chromium";v="129"',
            'sec-ch-ua-mobile': '?0',
            'sec-ch-ua-platform': '"Windows"',
            'sec-fetch-dest': 'script',
            'sec-fetch-mode': 'no-cors',
            'sec-fetch-site': 'same-site',
            'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.0.0 Safari/537.36'
        }
        # Send the GET request with headers and cookies
        cbox_response = requests.get(url, params=params, headers=headers, cookies=cookies)

        # Check if the request was successful
        cbox_response.raise_for_status()

        # Extract the cbox_token
        cbox_token = cbox_response.json()['result']['cbox_token']
        return cbox_token
    
    except requests.exceptions.RequestException as e:
        print(f"Error fetching cbox_token: {e}")
        return None
    except KeyError:
        print("Error: cbox_token not found in the response")
        return None

def _fetch_blog_no(blog_id, blog_post_id):
    url = f'https://blog.naver.com/PostView.naver?blogId={blog_id}&logNo={blog_post_id}&redirect=Dlog&widgetTypeCall=true&noTrackingCode=true&directAccess=false'
    
    try:
        res = requests.get(url)
        res.raise_for_status()  # Check if the request was successful
        
        content = res.content.decode('utf-8')
        match = re.search(r"blogNo\s*=\s*'(\d+)'", content)
        
        if match:
            return match.group(1)
        else:
            return "Blog Number not found"
    
    except requests.RequestException as e:
        return f"An error occurred: {e}"

def create_comment(cookies, blog_Num, blog_id, blog_post_id, cbox_token, comment_content):
    try:
        # Define the URL for creating a comment
        url_create = "https://apis.naver.com/commentBox/cbox/web_naver_create_json.json"

        # Define the query parameters
        params_create = {
            'ticket': 'blog',
            'templateId': 'default',
            'pool': 'blogid',
            '_cv': '20241007144116'
        }

        # Define the headers
        headers_create = {
            'accept': 'application/json, text/javascript, */*; q=0.01',
            'accept-encoding': 'gzip, deflate, br, zstd',
            'accept-language': 'ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7',
            'origin': 'https://m.blog.naver.com',
            'referer': f'https://m.blog.naver.com/CommentList.naver?blogId={blog_id}&logNo={blog_post_id}',
            'sec-ch-ua': '"Google Chrome";v="129", "Not=A?Brand";v="8", "Chromium";v="129"',
            'sec-ch-ua-mobile': '?0',
            'sec-ch-ua-platform': '"Windows"',
            'sec-fetch-dest': 'empty',
            'sec-fetch-mode': 'cors',
            'sec-fetch-site': 'same-site',
            'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.0.0 Safari/537.36',
            'content-type': 'application/x-www-form-urlencoded'
        }

        # Define the POST request body data
        
# https://apis.naver.com/commentBox/cbox/web_naver_create_json.json?ticket=blog&templateId=default&pool=blogid&_cv=20241007144116
        print(f'https://m.blog.naver.com/CommentList.naver?blogId={blog_id}&logNo={blog_post_id}')
        data = {
            'lang': 'ko',
            'country': '',
            'objectId': f'{blog_Num}_201_{blog_post_id}',
            'categoryId': '',
            'pageSize': '100',
            'indexSize': '10',
            'groupId': f'{blog_Num}',
            'listType': 'OBJECT',
            'pageType': 'more',
            'objectUrl': f'https://m.blog.naver.com/CommentList.naver?blogId={blog_id}&logNo={blog_post_id}',
            'contents': comment_content,
            'userType': '',
            'pick': 'false',
            'manager': 'false',
            'score': '0',
            'sort': 'NEW',
            'secret': 'false',
            'refresh': 'false',
            'imageCount': '0',
            'commentType': 'txt',
            'validateBanWords': 'false',
            'cbox_token': cbox_token
        }

        # Send the POST request
        response_create = requests.post(url_create, params=params_create, headers=headers_create, cookies=cookies, data=data)

        # Check if the request was successful
        response_create.raise_for_status()

        # Print the response status and text
        print(f"Response status code: {response_create.status_code}")
        print(f"Response text: {response_create.text}")
        return response_create.status_code, response_create.text

    except requests.exceptions.RequestException as e:
        print(f"Error creating the comment: {e}")
        return None, None


# Example usage
blog_id = "eerony"
blog_post_id = "223621827803"
comment_content = "저도 포스팅 잘 보고 갑니다 :) 방문해주셔서 감사합니다!"

blog_Num = _fetch_blog_no(blog_id, blog_post_id)
cbox_token = _get_cbox_token(cookies=cookies, blog_post_id=blog_post_id)

if cbox_token:
    # Step 2: Create a comment using the token
    status, response_text = create_comment(cookies=cookies, blog_Num=blog_Num, blog_id=blog_id, blog_post_id=blog_post_id, cbox_token=cbox_token, comment_content=comment_content)
else:
    print("Failed to retrieve cbox_token.")

https://m.blog.naver.com/CommentList.naver?blogId=eerony&logNo=223621827803
Response status code: 200
Response text: {"code":"1000","country":"KR","date":"2024-10-22T13:40:03+0000","lang":"ko","message":"요청을 성공적으로 처리하였습니다.","result":{"commentList":[{"ticket":"blog","objectId":"154352947_201_223621827803","categoryId":"*","templateId":"default","commentNo":"840653523342327825","parentCommentNo":"840653523342327825","replyLevel":1,"replyCount":0,"replyAllCount":0,"replyPreviewNo":null,"replyList":null,"imageCount":0,"imageList":null,"imagePathList":null,"imageWidthList":null,"imageHeightList":null,"commentType":"txt","stickerId":null,"sticker":null,"sortValue":1729604403850,"contents":"저도 포스팅 잘 보고 갑니다 :) 방문해주셔서 감사합니다!","userIdNo":"","exposedUserIp":null,"lang":"ko","country":"KR","idType":"naver","idProvider":"naver","userName":"대한광고","userProfileImage":"https://blogpfthumb-phinf.pstatic.net/MjAyNDA2MjdfMzcg/MDAxNzE5NDk1NDA1NTI4.bYXlOXLkjJVFzhIx8UOQ-6pxtkqwTvKiE6DLtTPTl40g.oh_FUgKbEtCT-7

In [4]:
cbox_token

'DeUIGG8W8HQSsrqPW_Y9X34SXwndjzcgXGKbZcQLtrEJa413fndRL3C3MYhMQQSn_h58399ffcnh4lGedZNiXg'